Now we parse the content of the JSON file with the QASM circuits

In [1]:
using JSON
using ITensors



In [2]:

# Hadamard Gate
function h_gate(sites, i::Int)
    N = length(sites)
    h_op = op("H", sites[i])
    for k in 1:N
        if k != i
            h_op = h_op * op("Id", sites[k])
        end
    end
    return MPO(h_op, sites)
end

# Rz Gate with angle θ
function rz_gate(sites, i::Int, θ::Float64)
    N = length(sites)
    rz_op = op("Rz", sites[i]; θ=θ)
    for k in 1:N
        if k != i
            rz_op = rz_op * op("Id", sites[k])
        end
    end
    return MPO(rz_op, sites)
end

# Rx Gate with angle θ
function rx_gate(sites, i::Int, θ::Float64)
    N = length(sites)
    rx_op = op("Rx", sites[i]; θ=θ)
    for k in 1:N
        if k != i
            rx_op = rx_op * op("Id", sites[k])
        end
    end
    return MPO(rx_op, sites)
end

# Ry Gate with angle θ
function ry_gate(sites, i::Int, θ::Float64)
    N = length(sites)
    ry_op = op("Ry", sites[i]; θ=θ)
    for k in 1:N
        if k != i
            ry_op = ry_op * op("Id", sites[k])
        end
    end
    return MPO(ry_op, sites)
end

# Rxx Gate with angle ϕ
function rxx_gate(sites, i::Int, j::Int, ϕ::Float64)
    N = length(sites)
    rxx_op = op("Rxx", sites[i], sites[j]; ϕ=ϕ)
    for k in 1:N
        if k != i && k != j
            rxx_op = rxx_op * op("Id", sites[k])
        end
    end
    return MPO(rxx_op, sites)
end

# Rzz Gate with angle ϕ
function rzz_gate(sites, i::Int, j::Int, ϕ::Float64)
    N = length(sites)
    rzz_op = op("Rzz", sites[i], sites[j]; ϕ=ϕ)
    for k in 1:N
        if k != i && k != j
            rzz_op = rzz_op * op("Id", sites[k])
        end
    end
    return MPO(rzz_op, sites)
end

# Ryy Gate with angle ϕ
function ryy_gate(sites, i::Int, j::Int, ϕ::Float64)
    N = length(sites)
    ryy_op = op("Ryy", sites[i], sites[j]; ϕ=ϕ)
    for k in 1:N
        if k != i && k != j
            ryy_op = ryy_op * op("Id", sites[k])
        end
    end
    return MPO(ryy_op, sites)
end

# CX (Controlled-X) Gate
function cx_gate(sites, control::Int, target::Int)
    N = length(sites)
    cx_op = op("CX", sites[control], sites[target])
    for k in 1:N
        if k != control && k != target
            cx_op = cx_op * op("Id", sites[k])
        end
    end
    return MPO(cx_op, sites)
end

# Projector onto |0⟩ (|0⟩⟨0|)
function proj_0_gate(sites, i::Int)
    N = length(sites)
    proj0_op = op("Proj0", sites[i])
    for k in 1:N
        if k != i
            proj0_op = proj0_op * op("Id", sites[k])
        end
    end
    return MPO(proj0_op, sites)
end

# Projector onto |1⟩ (|1⟩⟨1|)
function proj_1_gate(sites, i::Int)
    N = length(sites)
    proj1_op = op("Proj1", sites[i])
    for k in 1:N
        if k != i
            proj1_op = proj1_op * op("Id", sites[k])
        end
    end
    return MPO(proj1_op, sites)
end


proj_1_gate (generic function with 1 method)

In [3]:
function create_sites(n::Int)
    return [Index(2, "Qubit,i$i") for i in 1:n]
end
# check if the MPO creation was sucessful
function inspect_mpo(mpo::MPO, sites::Vector)
    N = length(sites) 

    println("\nManual Inspection of MPO:")
    for site_idx in 1:N
        println("Site $site_idx Operator:")
        println(mpo[site_idx])
    end
end

inspect_mpo (generic function with 1 method)

In [33]:
function parse_subcircuits(json_filename::String)
    json_data = JSON.parsefile(json_filename)
    
    subexp_dict = Dict{String, Vector{Dict{String, Any}}}()
    qubit_num_dict = Dict{String, Int}()
    
    for entry in json_data
        category = entry["Subexperiment"]
        qubit_num = entry["Qubit number"]
        qubit_range = entry["Qubit range"]
        index = entry["Subcircuit"]
        operations_raw = entry["Operations"]
        
        # offset = qubit_range[1] - 1

        # Ensure consistency of qubit numbers within the same category
        if haskey(qubit_num_dict, category)
            if qubit_num_dict[category] != qubit_num
                error("Inconsistent qubit numbers in category $category")
            end
        else
            qubit_num_dict[category] = qubit_num
        end
        
        processed_operations = Vector{Dict{String, Any}}()
        for op in operations_raw
            name = op["Name"]
            qubits = op["Qubits"]
            angle_array = op["Angle"]
            
            adjusted_qubits = [q + 1 for q in qubits]
            
            angle = if length(angle_array) == 1
                        angle_array[1]
                    else
                        NaN  
                    end
            
            processed_op = Dict{String, Any}(
                "Name" => name,
                "Angle" => angle,
                "Qubits" => adjusted_qubits
            )
            
            push!(processed_operations, processed_op)
        end
        
        subcircuit = Dict{String, Any}(
            "Subcircuit" => index + 1,
            "Operations" => processed_operations
        )
        
        # Add the subcircuit to the subexp_dict
        if haskey(subexp_dict, category)
            push!(subexp_dict[category], subcircuit)
        else
            subexp_dict[category] = [subcircuit]
        end
    end
    
    sorted_categories = sort(collect(keys(subexp_dict)))
    
    organized_data = Dict{String, Dict{String, Any}}()
    
    for category in sorted_categories
        sorted_subcircuits = sort(subexp_dict[category], by = sc -> sc["Subcircuit"])
        organized_data[category] = Dict{String, Any}(
            "Qubit number" => qubit_num_dict[category],
            "Subcircuits" => sorted_subcircuits
        )
    end
    
    return organized_data
end

#Fixed qubit range to always be [1, n_max], and fixed subcircuit index to start at 1

json_filename = "subcircuits.json"  
circuits_data = parse_subcircuits(json_filename);


In [ ]:
function build_mpo_sequence(sites, operations, basis_gates)
    """
    Builds a list of Matrix Product Operators (MPOs) based on the provided operations.
    """
    mpo_list = []

    for op_dict in operations
        gate_type = op_dict["Name"]
        qubits = op_dict["Qubits"]
        angle = get(op_dict, "Angle", NaN) 

        if gate_type in basis_gates
            if gate_type == "h"
                i = qubits[1]
                push!(mpo_list, h_gate(sites, i))

            elseif gate_type == "rx"
                θ = angle
                i = qubits[1]
                push!(mpo_list, rx_gate(sites, i, θ))

            elseif gate_type == "ry"
                θ = angle
                i = qubits[1]
                push!(mpo_list, ry_gate(sites, i, θ))

            elseif gate_type == "rz"
                θ = angle
                i = qubits[1]
                push!(mpo_list, rz_gate(sites, i, θ))

            elseif gate_type == "rxx"
                φ = angle
                i, j = qubits[1], qubits[2]
                push!(mpo_list, rxx_gate(sites, i, j, φ))

            elseif gate_type == "rzz"
                φ = angle
                i, j = qubits[1], qubits[2]
                push!(mpo_list, rzz_gate(sites, i, j, φ))

            elseif gate_type == "ryy"
                φ = angle
                i, j = qubits[1], qubits[2]
                push!(mpo_list, ryy_gate(sites, i, j, φ))

            elseif gate_type == "cx"
                control, target = qubits[1], qubits[2]
                push!(mpo_list, cx_gate(sites, control, target))

            end
    
            elseif gate_type == "measure"
                push!(mpo_list, ("measure", qubits[1])) 
        
        
        else
            error("Unsupported gate type: $gate_type. Supported gates are: $(join(basis_gates, ", ")), Measure")
        end
    end

    return mpo_list
end


function apply_measurement(rho, sites, index)
    M0 = proj_0_gate(sites, index)
    M1 = proj_1_gate(sites, index)
    return apply(apply(M0, rho), dag(M0)) - apply(apply(M1, rho), dag(M1))
end




# # New combined function to build MPO sequences and apply them to generate rhos
# function mpo_sequence_apply(circ_data, basis_gates)
#     all_mpo_sequences = Dict{String, Vector{Any}}()
#     all_rhos = Dict{String, Vector{MPO}}()
#     observables = Dict{String, MPO}()

#     for label in keys(circ_data)
#         n_qubits = circ_data[label]["Qubit number"]
#         sites_label = create_sites(n_qubits)

#         initial_psi = productMPS(sites_label, "0")
#         initial_rho = outer(initial_psi, initial_psi')

#         label_mpo_sequences = Vector{Any}()
#         label_rhos = Vector{MPO}()

#         os = OpSum()
#         for i in 1:n_qubits
#             os += "Z", i
#         end
#         Zmpo = MPO(os, sites_label)

#         for subcircuit in circ_data[label]["Subcircuits"]
#             operations = subcircuit["Operations"]
#             index = subcircuit["Subcircuit"]

#             mpo_sequence = build_mpo_sequence(sites_label, operations, basis_gates)
#             push!(label_mpo_sequences, mpo_sequence)
            
#             # now we apply the MPO sequence
#             rho = initial_rho
#             for mpo in mpo_sequence
#                 if typeof(mpo) != MPO && mpo[1] == "measure"
#                     qubit_index = mpo[2]
#                     rho = apply_measurement(rho, sites_label, qubit_index)
#                 elseif isa(mpo, MPO)
#                     rho = apply(apply(mpo, rho), dag(mpo))
#                 end
#             end 
#             println("Completed applying MPO sequence for subcircuit $label$index.")
#             push!(label_rhos, rho)
#         end

#         all_mpo_sequences[label] = label_mpo_sequences
#         all_rhos[label] = label_rhos
#         observables[label] = Zmpo
#     end

#     return (all_mpo_sequences, all_rhos, observables)
# end


# Modified mpo_sequence_apply function
function mpo_sequence_apply(circ_data, basis_gates)
    all_mpo_sequences = Dict{String, Vector{Any}}()
    all_rhos = Dict{String, Vector{MPO}}()

    total_qubits = sum(circ_data[label]["Qubit number"] for label in keys(circ_data))
    combined_sites = create_sites(total_qubits)
    

    os = OpSum()
    for i in 1:total_qubits
        os += "Z", i
    end
    Zmpo = MPO(os, combined_sites)
    #ORDERING IS IMPORTANT!! We may have problems with this later
    sorted_labels = sort(collect(keys(circ_data)))

    for label in sorted_labels
        n_qubits = circ_data[label]["Qubit number"]
    
        initial_psi = productMPS(combined_sites, "0")
        initial_rho = outer(initial_psi, initial_psi')

        label_mpo_sequences = Vector{Any}()
        label_rhos = Vector{MPO}()

        for subcircuit in circ_data[label]["Subcircuits"]
            operations = subcircuit["Operations"]
            index = subcircuit["Subcircuit"]

            # Build the MPO sequence for the current subcircuit
            mpo_sequence = build_mpo_sequence(combined_sites, operations, basis_gates)
            push!(label_mpo_sequences, mpo_sequence)
            
            # Apply the MPO sequence to the initial density matrix
            rho = initial_rho
            for mpo in mpo_sequence
                if typeof(mpo) != MPO && mpo[1] == "measure"
                    qubit_index = mpo[2]
                    rho = apply_measurement(rho, combined_sites, qubit_index)
                elseif isa(mpo, MPO)
                    rho = apply(apply(mpo, rho; maxdim = 10, cutoff = 1e-4), dag(mpo); maxdim = 10, cutoff = 1e-4)
                end
            end 
            println("Completed applying MPO sequence for subcircuit $label$index.")
            push!(label_rhos, rho)
        end

        # Store the MPO sequences, rhos, and observables for the current label
        all_mpo_sequences[label] = label_mpo_sequences
        all_rhos[label] = label_rhos

    end

    return (all_mpo_sequences, all_rhos, Zmpo)
end


basis_gates = ["h", "rx", "ry", "rz", "rxx", "rzz", "ryy", "cx"]
all_mpo_sequences, all_rhos, observable = mpo_sequence_apply(circuits_data,basis_gates);


Contraction resulted in ITensor with 14 indices, which is greater
            than or equal to the ITensor order warning threshold 14.
            You can modify the threshold with macros like `@set_warn_order N`,
            `@reset_warn_order`, and `@disable_warn_order` or functions like
            `ITensors.set_warn_order(N::Int)`, `ITensors.reset_warn_order()`, and
            `ITensors.disable_warn_order()`.

Stacktrace:
  [1] _contract(A::ITensor, B::ITensor)
    @ ITensors C:\Users\pably\.julia\packages\ITensors\4M2ep\src\tensor_operations\tensor_algebra.jl:20
  [2] contract(A::ITensor, B::ITensor)
    @ ITensors C:\Users\pably\.julia\packages\ITensors\4M2ep\src\tensor_operations\tensor_algebra.jl:104
  [3] *(A::ITensor, B::ITensor)
    @ ITensors C:\Users\pably\.julia\packages\ITensors\4M2ep\src\tensor_operations\tensor_algebra.jl:91
  [4] rxx_gate(sites::Vector{Index{Int64}}, i::Int64, j::Int64, ϕ::Float64)
    @ Main .\In[2]:56
  [5] build_mpo_sequence(sites::Vector{Index{In

Excessive output truncated after 524330 bytes.

Contraction resulted in ITensor with 17 indices, which is greater
            than or equal to the ITensor order warning threshold 14.
            You can modify the threshold with macros like `@set_warn_order N`,
            `@reset_warn_order`, and `@disable_warn_order` or functions like
            `ITensors.set_warn_order(N::Int)`, `ITensors.reset_warn_order()`, and
            `ITensors.disable_warn_order()`.

Stacktrace:
  [1]

In [27]:
print(all_rhos["A"][1])
print(all_rhos["B"][1])

MPO
[1] ((dim=2|id=864|"Qubit,i1")', (dim=2|id=864|"Qubit,i1"), (dim=4|id=843|"Link,n=1"))
[2] ((dim=2|id=431|"Qubit,i2")', (dim=2|id=431|"Qubit,i2"), (dim=16|id=87|"Link,n=2"), (dim=4|id=843|"Link,n=1"))
[3] ((dim=2|id=16|"Qubit,i3")', (dim=2|id=16|"Qubit,i3"), (dim=16|id=490|"Link,n=3"), (dim=16|id=87|"Link,n=2"))
[4] ((dim=2|id=886|"Qubit,i4")', (dim=2|id=886|"Qubit,i4"), (dim=4|id=549|"Link,n=4"), (dim=16|id=490|"Link,n=3"))
[5] ((dim=2|id=575|"Qubit,i5")', (dim=2|id=575|"Qubit,i5"), (dim=4|id=549|"Link,n=4"))
MPO
[1] ((dim=2|id=659|"Qubit,i6")', (dim=2|id=659|"Qubit,i6"), (dim=4|id=708|"Link,n=1"))
[2] ((dim=2|id=133|"Qubit,i7")', (dim=2|id=133|"Qubit,i7"), (dim=16|id=589|"Link,n=2"), (dim=4|id=708|"Link,n=1"))
[3] ((dim=2|id=302|"Qubit,i8")', (dim=2|id=302|"Qubit,i8"), (dim=16|id=182|"Link,n=3"), (dim=16|id=589|"Link,n=2"))
[4] ((dim=2|id=52|"Qubit,i9")', (dim=2|id=52|"Qubit,i9"), (dim=4|id=825|"Link,n=4"), (dim=16|id=182|"Link,n=3"))
[5] ((dim=2|id=525|"Qubit,i10")', (dim=2|id=5

In [46]:
maxdim1 = 10
cutoff1 = 1e-12
coefs = [0.9605304970014426, 0.1947091711543252, -0.1947091711543252, 0.1947091711543252, -0.1947091711543252, 0.03946950299855745]
# final_rho = sum(coefs[i] * contract(all_rhos["A"][i], all_rhos["B"][i]; maxdim=maxdim1, cutoff=cutoff1) for i in 1:length(coefs))
final_rho = sum(coefs[i] * all_rhos["A"][i] * all_rhos["B"][i]' for i in 1:length(coefs))

MPO
[1] ((dim=2|id=199|"Qubit,i1"), (dim=2|id=199|"Qubit,i1")'', (dim=2|id=233|"Link,n=1"))
[2] ((dim=2|id=322|"Qubit,i2"), (dim=2|id=322|"Qubit,i2")'', (dim=4|id=314|"Link,n=2"), (dim=2|id=233|"Link,n=1"))
[3] ((dim=2|id=368|"Qubit,i3"), (dim=2|id=368|"Qubit,i3")'', (dim=4|id=473|"Link,n=3"), (dim=4|id=314|"Link,n=2"))
[4] ((dim=2|id=85|"Qubit,i4"), (dim=2|id=85|"Qubit,i4")'', (dim=2|id=17|"Link,n=4"), (dim=4|id=473|"Link,n=3"))
[5] ((dim=2|id=482|"Qubit,i5"), (dim=2|id=482|"Qubit,i5")'', (dim=1|id=430|"Link,n=5"), (dim=2|id=17|"Link,n=4"))
[6] ((dim=2|id=811|"Qubit,i6"), (dim=2|id=811|"Qubit,i6")'', (dim=2|id=953|"Link,n=6"), (dim=1|id=430|"Link,n=5"))
[7] ((dim=2|id=479|"Qubit,i7"), (dim=2|id=479|"Qubit,i7")'', (dim=4|id=86|"Link,n=7"), (dim=2|id=953|"Link,n=6"))
[8] ((dim=2|id=385|"Qubit,i8"), (dim=2|id=385|"Qubit,i8")'', (dim=4|id=75|"Link,n=8"), (dim=4|id=86|"Link,n=7"))
[9] ((dim=2|id=535|"Qubit,i9"), (dim=2|id=535|"Qubit,i9")'', (dim=2|id=87|"Link,n=9"), (dim=4|id=75|"Link,n=8"

In [47]:
final_rho = replaceprime(final_rho, 2 => 1)

MPO
[1] ((dim=2|id=199|"Qubit,i1"), (dim=2|id=199|"Qubit,i1")', (dim=2|id=233|"Link,n=1"))
[2] ((dim=2|id=322|"Qubit,i2"), (dim=2|id=322|"Qubit,i2")', (dim=4|id=314|"Link,n=2"), (dim=2|id=233|"Link,n=1"))
[3] ((dim=2|id=368|"Qubit,i3"), (dim=2|id=368|"Qubit,i3")', (dim=4|id=473|"Link,n=3"), (dim=4|id=314|"Link,n=2"))
[4] ((dim=2|id=85|"Qubit,i4"), (dim=2|id=85|"Qubit,i4")', (dim=2|id=17|"Link,n=4"), (dim=4|id=473|"Link,n=3"))
[5] ((dim=2|id=482|"Qubit,i5"), (dim=2|id=482|"Qubit,i5")', (dim=1|id=430|"Link,n=5"), (dim=2|id=17|"Link,n=4"))
[6] ((dim=2|id=811|"Qubit,i6"), (dim=2|id=811|"Qubit,i6")', (dim=2|id=953|"Link,n=6"), (dim=1|id=430|"Link,n=5"))
[7] ((dim=2|id=479|"Qubit,i7"), (dim=2|id=479|"Qubit,i7")', (dim=4|id=86|"Link,n=7"), (dim=2|id=953|"Link,n=6"))
[8] ((dim=2|id=385|"Qubit,i8"), (dim=2|id=385|"Qubit,i8")', (dim=4|id=75|"Link,n=8"), (dim=4|id=86|"Link,n=7"))
[9] ((dim=2|id=535|"Qubit,i9"), (dim=2|id=535|"Qubit,i9")', (dim=2|id=87|"Link,n=9"), (dim=4|id=75|"Link,n=8"))
[10] (

In [48]:
exp_val_knitting = inner(observable, final_rho)
print(exp_val_knitting)

1.8553997037914653 + 9.992007221626409e-16im

In [9]:
### SOME TESTS

n_qubits = 3
sites = create_sites(n_qubits)
gates_test = [(:h, i) for i in 1:n_qubits]
push!(gates_test, (:measure_qpd, 1))
mpos_test = build_mpo_sequence(sites, gates_test);

psi = productMPS(sites, "0")
rho = outer(psi,psi')


Z_sum_op = ITensor()

os = OpSum()
for i in 1:n_qubits
    os += "X", i
end
Zmpo = MPO(os, sites)

rho1 = apply(apply(mpos_test[1], rho), mpos_test[1])
for mpo in mpos_test[2:end]
    # Ensure the MPO and MPS share the same site indices
    rho1 = rho1 + apply(apply(mpo, rho), dag(mpo))
end

LoadError: MethodError: no method matching build_mpo_sequence(::Vector{Index{Int64}}, ::Vector{Tuple{Symbol, Int64}})

[0mClosest candidates are:
[0m  build_mpo_sequence(::Any, ::Any, [91m::Any[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[5]:1[24m[39m


In [ ]:
inner(rho, Zmpo)

0.0